## This webscraping script grabs current margin rates from the TradeStation website, loads it into a dataframe, converts strings to numbers, and provides interface to access data.

## Below is a screen shot of a populated dataframe.  The code below can be executed by typing shift-return in each code cell below.  Code to export data to excel is provided, but will not work from a shared resource like Binder, must run locally.

![title](margin_rates_df.png)

In [1]:
from gazpacho import get, Soup
import pandas as pd
#pd.options.mode.chained_assignment = None  # default='warn'
#pd.set_option("display.precision", 0)

In [2]:
url = 'https://www.tradestation.com/pricing/futures-margin-requirements/'
response = get(url)
soup = Soup(response)
df = pd.read_html(str(soup.find('table')))[0]

In [3]:
df.rename(columns={'Symbol Root': 'Symbol', 'Initial Margin': 'InitialMargin', 'Maint. Margin': 'MaintMargin','Day Trading Rate': 'DayTradeRate'}, inplace=True)

In [4]:
df1 = df[df['InitialMargin'].str.startswith('$')].copy(deep=False)
df1.set_index('Symbol',inplace=True)

In [5]:
df1['InitialMargin'] = df1['InitialMargin'].str.replace('$','')
df1['InitialMargin'] = df1['InitialMargin'].str.replace(',','')
df1.InitialMargin = pd.to_numeric(df1.InitialMargin)
df1['DayTradeMargin'] = df1.apply(lambda x: x['InitialMargin']*0.25 if x['DayTradeRate'] == '25% of initial' else 0, axis=1)
df1['DayTradeMargin'] = df1.apply(lambda x: x['InitialMargin']*0.50 if x['DayTradeRate'] == '50% of initial' else 0, axis=1)
df1['InitialMargin'] = pd.to_numeric(df1['InitialMargin']).apply(lambda x: "${:,}".format(x))
df1['DayTradeMargin'] = pd.to_numeric(df1['DayTradeMargin']).apply(lambda x: "${0:,.2f}".format(x))

In [6]:
df1.to_excel("marginrates.xlsx")

In [7]:
df1.loc['ES']['InitialMargin']

'$13,200'

In [8]:
df1

,Description,InitialMargin,MaintMargin,DayTradeRate,DayTradeMargin
Symbol,,,,,
ES,E-MINI S&P 500,"$13,200","$12,000",25% of initial,$0.00
EMD,E-MINI MIDCAP 400,"$18,700","$17,000",NONE,$0.00
NQ,E-MINI NASDAQ 100,"$16,500","$15,000",25% of initial,$0.00
RTY,MINI RUSSELL 2000 (CME),"$7,810","$7,100",25% of initial,$0.00
YM,MINI DOW JONES ($5),"$13,200","$12,000",25% of initial,$0.00
...,...,...,...,...,...
CT,COTTON,"$2,915","$2,650",NONE,$0.00
OJ,FROZEN OJ,"$1,386","$1,260",NONE,$0.00
CC,COCOA,"$2,090","$1,900",NONE,$0.00
